### **Objective:**

You want a user to be able to invoke the following function call organically as part of a dialog exchange:

```python
get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
    "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
```

`RExtract` is provided such that the following knowledge base syntax can be used:
```python
known_info = KnowledgeBase()
extractor = RExtract(KnowledgeBase, InstructLLM(), parser_prompt)
results = extractor.invoke({'info_base' : known_info, 'input' : 'My message'})
known_info = results['info_base']
```

**Design a chatbot that implements the following features:**
- The bot should start off by making small-talk, possibly helping the user with non-sensitive queries which don't require any private info access.
- When the user starts to ask about things that are database-walled (both practically and legally), tell the user that they need to provide the relevant information.
- When the retrieval is successful, the agent will be able to talk about the database-walled information.

**This can be done with a variety of techniques, including the following:**
- **Prompt Engineering and Context Parsing**, where the overall chat prompt stays roughly the same but the context is manipulated to to change agent behavior. For example, a failed db retrieval could be changed into an injection of natural-language instructions for how to resolve the problem such as *`"Information could not be retrieved with keys {...}. Please ask the user for clarification or help them with known information."`*
- **"Prompt Passing,"** where the active prompts are passed around as state variables and can be overridden by monitoring chains.
- **Branching chains** such as [**`RunnableBranch`**](https://python.langchain.com/docs/expression_language/how_to/routing) or more custom solutions that implement an conditional routing mechanism.
    - In the case of [`RunnableBranch`](https://python.langchain.com/docs/expression_language/how_to/routing), a `switch` syntax of the style:
        ```python
        from langchain.schema.runnable import RunnableBranch
        RunnableBranch(
            ((lambda x: 1 in x), RPrint("Has 1 (didn't check 2): ")),
            ((lambda x: 2 in x), RPrint("Has 2 (not 1 though): ")),
            RPrint("Has neither 1 not 2: ")
        ).invoke([2, 1, 3]);  ## -> Has 1 (didn't check 2): [2, 1, 3]
        ```

Some prompts and a gradio loop are provided that might help with the effort, but the agent will currently just hallucinate! Please implement the internal chain to try and retrieve the relevant information. Before trying to implement, look over the default behavior of the model and note how it might hallucinate or forget things.

In [ ]:
## TODO: Pick a working model that you're happy with
instruct_llm = (ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())
chat_model = (ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())

## The external chain is already supported, and just streams the output
external_chain = external_prompt | chat_model

## The internal chain is invoked in the loop, but is not defined yet

#####################################################################################
## START TODO: Define the extractor and internal chain to satisfy the objective
extractor = RunnableAssign({'know_base' : RExtract(KnowledgeBase, instruct_llm, parser_prompt)})
internal_chain = extractor | RunnableAssign(
    {'context' : lambda d: itemgetter('know_base') | get_key | get_flight_info}
)

## END TODO
#####################################################################################